In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import csv
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
import string
from nltk import word_tokenize
import gensim 
from gensim.models import Word2Vec 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")
import numpy as np
import pandas as pd
import pickle



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
original=[]
changed = []
with open('/content/drive/MyDrive/IR Project/archive/Questions.csv', encoding="latin1") as csvfile ,open('/content/drive/MyDrive/IR Project/archive/Output.csv' ,'w' ,newline='') as myfile:
  readCSV = csv.reader(csvfile, delimiter=',' )
  writeCSV = csv.writer(myfile , delimiter = ',')
  next(readCSV, None) 
  for row in readCSV:
    doc_id = row[0];
    title = row[5];
    original.append(title)
    data = preprocessing(title)
    replace = row[5].replace(title,data)
    row[5] = replace
    changed.append(row[5])
    writeCSV.writerow({row[5]})
    


In [ ]:
for doc in questions:
  w =doc.split()
  print(w)

In [3]:
stop_words=stopwords.words('english')
stemming = PorterStemmer()

In [4]:
def preprocessing(text):
  text = text.lower()
  text = text.replace("?"," ?")
  text = text.split()
  words = [stemming.stem(w) for w in text if not w in stop_words]
  sentence  = ' '.join(words)
  return sentence
  




In [ ]:
with open('/content/drive/MyDrive/IR Project/archive/Output.csv' ,'w' ,newline='') as myfile:
  writeCSV = csv.writer(myfile , delimiter = ',')
  for w in changed:
    writeCSV.writerow({ w })


In [ ]:
changed = np.array(changed)

In [ ]:
np.savetxt("Output.csv",  
           changed, 
           delimiter =", ",  
           fmt ='% s') 

In [ ]:
changed = list(changed)

In [5]:
changed = []
with open('/content/drive/MyDrive/IR project/Output.csv', encoding="latin1") as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',' )
  for row in readCSV:
    title = row[0];
    changed.append(title)




In [6]:
!pip install gensim

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
model_w2v = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/IR Project/Pre-Trained model/SO_vectors_200.bin", binary=True)

In [ ]:
filename = "/content/drive/MyDrive/IR Project/W2V_model.sav"
pickle.dump(model_w2v, open(filename, 'wb'))

In [7]:
model_w2v = pickle.load(open("/content/drive/MyDrive/IR project/W2V_model.sav", 'rb'))

In [ ]:
#model_w2v = gensim.models.Word2Vec(words , size=300, window=10 , negative=25)
#model_w2v.train(words, total_examples=len(words), epochs=20)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidfModel(sentences):
  tfidf_model = TfidfVectorizer()
  tfidf_model.fit(sentences)
  word_Tfidf_Dict = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
  tfidf_features = tfidf_model.get_feature_names()
  return word_Tfidf_Dict, tfidf_features

In [9]:
dictionary, features = tfidfModel(changed)


In [90]:
def tfidfWeightedWord2vec(sentences, dictionary, features):
  x_train = []
  w2v_vocab = list(model_w2v.wv.vocab)
  for sent in sentences:
    #print(sent)
    sent_vec = np.zeros(200)
    ti_idf_score_sum = 0 
    wordlist = sent.split(" ")
    #print(wordlist)
    for word in wordlist:
      if word in w2v_vocab and word in features:
        word_vec = model_w2v.wv[word]
        tf_idf_score = dictionary[word]*(wordlist.count(word)/len(wordlist))
        sent_vec += (word_vec * tf_idf_score)
        ti_idf_score_sum += tf_idf_score 
    if (ti_idf_score_sum !=0):
      sent_vec = sent_vec/ti_idf_score_sum
    x_train.append(sent_vec) 
  return x_train  

In [ ]:
sliced_data = changed

In [ ]:
x_train = tfidfWeightedWord2vec(sliced_data, dictionary, features)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df = pd.DataFrame(x_train)
df.shape

In [ ]:
df.to_csv("/content/drive/MyDrive/IR project/combined/w2vembeddings_full_combined.csv" ,header=None, index=False)

In [11]:
def readCsv(path):
  with open(path, encoding="latin1") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',' )
    test_data_embeddings = []
    for row in readCSV:
      embedd = []
      for i in range(0,len(row)):
        embedd.append(row[i])
      embedd = [(float)(word_embedding) for word_embedding in embedd]
      test_data_embeddings.append(embedd)   
  return test_data_embeddings

In [12]:
def getTitleList():
  titleList = []
  with open('/content/drive/MyDrive/IR project/Questions.csv', encoding="latin1") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',' )
    next(readCSV, None) 
    for row in readCSV:
      title = row[5];
      titleList.append(title)
  return titleList    


In [13]:
titlelist = getTitleList()

In [78]:
import numpy as np
from scipy.spatial import distance
def calCosSim(test_query_embedding, embeddings, titlelist):
  test_query_embeddingArr = np.array(test_query_embedding)
  embeddingsArr = np.array(embeddings)
  result = distance.cdist(test_query_embeddingArr, embeddingsArr, "cosine")
  final_res = 1-result[0]
  final_list = list(zip(final_res, titlelist))
  final_list.sort(reverse=True)
  return final_list


In [20]:
train_embeddings = readCsv("/content/drive/MyDrive/IR project/combined/w2vembeddings_full_combined.csv")

In [50]:
print(train_embeddings[0])

[0.20052852835320084, 0.7336173834239724, -1.0453705266423161, 2.3215986288581436, -0.31390727770310894, 0.7074048479329957, 0.7012878087270558, -0.5686441048762712, -0.5200858870098016, 0.12886488413520406, -0.19329062646556783, -1.1943364387535003, 0.79344214032542, -0.45816649287241296, 0.16804326207553733, 0.6293142736762547, -0.03141296339781829, -0.6453076616875263, -0.5186502010446434, 0.06723510531003012, -1.1899817382821094, -3.142119780586133, 1.1117615923945154, 1.6613509834069562, -0.6035607034189209, 0.6522275947174854, 0.00023530728134137828, -0.15453850531424962, -1.1309983819024794, 0.3981271726181166, 1.174100076924874, 0.24799571052188926, -1.0760593482131156, -1.4377396165277019, -1.635493265678283, 0.3745747025213205, -0.1949189382719861, 0.8625509849925447, -0.5867752981620665, -0.22973177939169967, -0.7011575500513022, 1.6495715111317202, -1.439697854011907, 0.994622263601535, 1.1674021713399447, -1.48184971820473, -1.962956409601736, 1.8973748127002508, -0.390482

In [93]:
queries = ['insert query in sql table', 'tags in html and css', 'concatenate a string in C#' ]
for query in queries:
  processed_test_query = preprocessing(query)
  print("query: ", query)
  test_query_embeddings = tfidfWeightedWord2vec([processed_test_query], dictionary, features)
  final_list = calCosSim(test_query_embeddings, train_embeddings, titlelist)
  print("relevant result with score:")
  for f in final_list[:10]:
    print(f[0]," : ", f[1])
  print("\n")

query:  insert query in sql table


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':


relevant result with score:
0.9324066561222449  :  sql - insert into multiple tables in one query
0.8910677098747644  :  Is it possible in boost::ireplace to treat special characters like basic characters? (eg. 'Åº' as 'z')
0.8905303864712878  :  MySQL INSERT ...SELECT with SUBQUERY
0.8767070552551252  :  Error in downloading.... The archived file is corrupted or damaged
0.8707882927629336  :  Rails 3 CSV imports to raise errors in resulting view
0.8631997106588076  :  How to display page loading GIF on page with <frameset>?
0.8629816151076443  :  SQL Delete (Suspended in activity monitor)
0.8625800477254546  :  iOS (Xcode 6.2) Paypal (2.11.0) integration - 64 duplicate symbols for architecture arm64
0.8625800477254545  :  How to update an SQL table with F#
0.8567235539070933  :  has_many relation works on localhost but not server (Heroku)


query:  tags in html and css
relevant result with score:
0.9293471065464471  :  Empty HTML tags
0.9039089091488283  :  Importing text data with va